In [1]:
!pip install tfx

In [2]:
!pip install keras-tuner==1.0.4

In [3]:
import tensorflow as tf
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import os
import pprint
pp = pprint.PrettyPrinter()
import zipfile

In [4]:
local = '/content/metro+interstate+traffic+volume.zip'
zipref = zipfile.ZipFile(local)
zipref.extractall('/content/data')

In [6]:
import gzip
import shutil

# Input and output filenames
input_file = "/content/data/Metro_Interstate_Traffic_Volume.csv.gz"
output_file = "/content/data/metro_data/metro.csv"


# Decompress the file
with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"Converted {input_file} to {output_file}")

Converted /content/data/Metro_Interstate_Traffic_Volume.csv.gz to /content/data/metro_data/metro.csv


In [7]:
path = '/content/pipeline'
if not os.path.exists(path):
  _pipeline_root = os.mkdir(path)

_data_root = '/content/data/metro_data'

_data_filepath = os.path.join(_data_root,'metro.csv')

print(os.listdir('/content'))

['.config', 'metro+interstate+traffic+volume.zip', 'pipeline', 'data', 'sample_data']


In [8]:
!head {_data_filepath}

holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918
None,291.72,0.0,0.0,1,Clear,sky is clear,2012-10-02 14:00:00,5181
None,293.17,0.0,0.0,1,Clear,sky is clear,2012-10-02 15:00:00,5584
None,293.86,0.0,0.0,1,Clear,sky is clear,2012-10-02 16:00:00,6015
None,294.14,0.0,0.0,20,Clouds,few clouds,2012-10-02 17:00:00,5791


In [9]:
context = InteractiveContext(pipeline_root = '/content/pipeline')

In [10]:
example_gen = tfx.components.CsvExampleGen(input_base = _data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
# get the artifact of the examplegen which stores the split and uri
artifacts = example_gen.outputs['examples'].get()[0]
print(f"Split names : {artifacts.split_names}")
print(f"uri : {artifacts.uri}")

Split names : ["train", "eval"]
uri : /content/pipeline/CsvExampleGen/examples/1


In [12]:
# Load the split in the environment
train_uri = os.path.join(artifacts.uri,"Split-train")
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [13]:
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
context.show(statistics_gen.outputs['statistics'])

In [14]:
# Get files in the directory
tfrecord_file = [os.path.join(train_uri,name)for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_file,compression_type = 'GZIP')

In [15]:
# Tf.train.example ()
import tensorflow as tf

# Create a dictionary of features
feature_dict = {
    'name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'John'])),
    'age': tf.train.Feature(int64_list=tf.train.Int64List(value=[30])),
    'score': tf.train.Feature(float_list=tf.train.FloatList(value=[85.5])),
}

# Create a Features object from the feature dictionary
features = tf.train.Features(feature=feature_dict)

# Create an Example using the Features object
example = tf.train.Example(features=features)
example

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 30
      }
    }
  }
  feature {
    key: "name"
    value {
      bytes_list {
        value: "John"
      }
    }
  }
  feature {
    key: "score"
    value {
      float_list {
        value: 85.5
      }
    }
  }
}

In [16]:
# Individual features and values

def feature_value_dis():
  records = []
  # Looping to the dataset
  for i in dataset.take(3):
    # get the numpy array of tensor
    serialized_example = i.numpy()
    # Protcol buffer message used to storing and exchanging structure data
    example = tf.train.Example()
    # read the example
    example.ParseFromString(serialized_example)
    # Convert the message into python dictionary
    example_dict = (MessageToDict(example))
    records.append(example_dict)
  return records

feature_value_dis()

[{'features': {'feature': {'date_time': {'bytesList': {'value': ['MjAxMi0xMC0wMiAxMTowMDowMA==']}},
    'holiday': {'bytesList': {'value': ['Tm9uZQ==']}},
    'clouds_all': {'int64List': {'value': ['90']}},
    'weather_main': {'bytesList': {'value': ['Q2xvdWRz']}},
    'traffic_volume': {'int64List': {'value': ['4767']}},
    'temp': {'floatList': {'value': [289.58]}},
    'weather_description': {'bytesList': {'value': ['b3ZlcmNhc3QgY2xvdWRz']}},
    'rain_1h': {'floatList': {'value': [0.0]}},
    'snow_1h': {'floatList': {'value': [0.0]}}}}},
 {'features': {'feature': {'rain_1h': {'floatList': {'value': [0.0]}},
    'temp': {'floatList': {'value': [290.13]}},
    'snow_1h': {'floatList': {'value': [0.0]}},
    'clouds_all': {'int64List': {'value': ['90']}},
    'weather_main': {'bytesList': {'value': ['Q2xvdWRz']}},
    'traffic_volume': {'int64List': {'value': ['5026']}},
    'date_time': {'bytesList': {'value': ['MjAxMi0xMC0wMiAxMjowMDowMA==']}},
    'holiday': {'bytesList': {'valu

In [17]:
# schema generation

schema_gen = tfx.components.SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [22]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'clouds_all',INT,required,,-
'date_time',BYTES,required,,-
'holiday',STRING,required,,'holiday'
'rain_1h',FLOAT,required,,-
'snow_1h',FLOAT,required,,-
'temp',FLOAT,required,,-
'traffic_volume',INT,required,,-
'weather_description',STRING,required,,'weather_description'
'weather_main',STRING,required,,'weather_main'


,Values
Domain,
'holiday',"'Christmas Day', 'Columbus Day', 'Independence Day', 'Labor Day', 'Martin Luther King Jr Day', 'Memorial Day', 'New Years Day', 'None', 'State Fair', 'Thanksgiving Day', 'Veterans Day', 'Washingtons Birthday'"
'weather_description',"'SQUALLS', 'Sky is Clear', 'broken clouds', 'drizzle', 'few clouds', 'fog', 'freezing rain', 'haze', 'heavy intensity drizzle', 'heavy intensity rain', 'heavy snow', 'light intensity drizzle', 'light intensity shower rain', 'light rain', 'light rain and snow', 'light shower snow', 'light snow', 'mist', 'moderate rain', 'overcast clouds', 'proximity shower rain', 'proximity thunderstorm', 'proximity thunderstorm with drizzle', 'proximity thunderstorm with rain', 'scattered clouds', 'shower drizzle', 'shower snow', 'sky is clear', 'sleet', 'smoke', 'snow', 'thunderstorm', 'thunderstorm with drizzle', 'thunderstorm with heavy rain', 'thunderstorm with light drizzle', 'thunderstorm with light rain', 'thunderstorm with rain', 'very heavy rain'"
'weather_main',"'Clear', 'Clouds', 'Drizzle', 'Fog', 'Haze', 'Mist', 'Rain', 'Smoke', 'Snow', 'Squall', 'Thunderstorm'"


In [18]:
# Example validator

example_validator= tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = schema_gen.outputs['schema'],
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [19]:
context.show(example_validator.outputs['anomalies'])

In [20]:
# constant module file
_metro_constant_module_file = "metro_constant.py"

In [26]:
%%writefile {_metro_constant_module_file}

# feature with string data type
CATEGORICAL_FEATURE_KEYS = [
  'weather_main','holiday','date_time'
]

# Feature with numerical data type
NUMERICAL_FEATURE_KEY = [
  'clouds_all','traffic_volume'
]


# Utility function
def transformed_name(key):
  return key + '_xf'

Overwriting metro_constant.py


In [27]:
# use tensorflow transform to transform these features
_metro_transform_module_file = "metro_transform.py"

In [48]:
%%writefile {_metro_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import metro_constant

_NUMERICAL_FEATURE_KEY = metro_constant.NUMERICAL_FEATURE_KEY
_CATEGORICAL_FEATURE_KEY = metro_constant.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEY = metro_constant.BUCKET_FEATURE_KEY
_FEATURE_BUCKET_COUNT = metro_constant.FEATURE_BUCKET_COUNT
_LABEL = metro_constant.LABEL_KEY
_transformed_name = metro_constant.transformed_name

# define the transformation
def preprocessing_fn(input):
  output = {}
  # Features to range from 0 to 1
  for i in _NUMERICAL_FEATURE_KEY:
    output[_transformed_name] = tft.scale_to_0_1(input[i])

  # convert the categorical to vocabulary
  for i in _CATEGORICAL_FEATURE_KEY:
    output[_transformed_name] = tft.compute_and_apply_vocabulary(input[i])



  return output

Overwriting metro_transform.py


In [50]:
# Inititate transform component

# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

transform_gen = tfx.components.Transform(
    examples = example_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(_metro_transform_module_file)
)

context.run(transform_gen)

TypeError: ignored